In [1]:
from kaggle_datasets import KaggleDatasets
!pip install ../input/shopee-external-models/Keras_Applications-1.0.8-py3-none-any.whl
!pip install ../input/shopee-external-models/efficientnet-1.1.0-py3-none-any.whl
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
import cudf
import cuml
import cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml import PCA
from cuml.neighbors import NearestNeighbors
import tensorflow as tf
import efficientnet.tfkeras as efn
from tqdm.notebook import tqdm
import math
from shutil import copyfile

copyfile(src = "../input/shopee-external-models/tokenization.py", dst = "../working/tokenization.py")
import tokenization
import tensorflow_hub as hub

Processing /kaggle/input/shopee-external-models/Keras_Applications-1.0.8-py3-none-any.whl
Processing /kaggle/input/shopee-external-models/efficientnet-1.1.0-py3-none-any.whl


In [2]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
BATCH_SIZE = 8
IMAGE_SIZE = [512, 512]
# Seed
SEED = 42
# Verbosity
VERBOSE = 1
"""Settings"""
# Number of classes
N_CLASSES = 11014
#how to ensemble matchs result 
CONCAT=True
#image_model_weight_path0='../input/shopee-adacos-fold/eff_adacos_model_fold0.h5'
image_model_weight_path1='../input/effnetb3adacos/EfficientNetB3_adacos_512_42.h5'
image_model_weight_path2='../input/effnetb4adacos/EfficientNetB4_adacos_512_42.h5'
#image_model_weight_path3='../input/shopee-adacos-fold/eff_adacos_model_fold3.h5'
image_model_weight_sam='../input/shopepeeffnetadacossam/EfficientNetB5_sam_adacos_512_42 (1).h5'
text_model_weight_path0='../input/shopee-bert-adacos/bert-arcface.h5'
# Flag to get cv score
GET_CV = False
image_output_layer=-5

In [3]:
# RESTRICT TENSORFLOW TO 2GB OF GPU RAM
# SO THAT WE HAVE 14GB RAM FOR RAPIDS
LIMIT = 8.0
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        #print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
print('We will restrict TensorFlow to max %iGB GPU RAM'%LIMIT)
print('then RAPIDS can use %iGB GPU RAM'%(16-LIMIT))

We will restrict TensorFlow to max 8GB GPU RAM
then RAPIDS can use 8GB GPU RAM


In [4]:

# Flag to check ram allocations (debug)
CHECK_SUB = False
from tensorflow.keras import backend as K
df = pd.read_csv('../input/shopee-product-matching/test.csv')
# If we are comitting, replace train set for test set and dont get cv
if len(df) > 3:
    GET_CV = False
del df

In [5]:
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    
def get_valid():
    seed_everything(SEED)
    print('\n')
    print('-'*50)
    train, valid = train_test_split(TRAINING_FILENAMES, shuffle = True, random_state = SEED)
    return valid

In [6]:
import math
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras.initializers import Constant
from tensorflow.python.keras.utils import tf_utils


def _resolve_training(layer, training):
    if training is None:
        training = K.learning_phase()
    if isinstance(training, int):
        training = bool(training)
    if not layer.trainable:
        # When the layer is not trainable, override the value
        training = False
    return training#tf_utils.constant_value(training)

In [7]:
import tensorflow.keras.layers
class AdaCos(Layer):
    """
    Implementation of AdaCos layer. Reference: https://arxiv.org/abs/1905.00292
    
    Arguments:
      num_classes: number of classes to classify
      is_dynamic: if False, use Fixed AdaCos. Else, use Dynamic Adacos.
      regularizer: weights regularizer
    """
    def __init__(self,
                 num_classes,
                 is_dynamic=True,
                 regularizer=None,
                 **kwargs):

        super().__init__(**kwargs)
        self._n_classes = num_classes
        self._init_s = math.sqrt(2) * math.log(num_classes - 1)
        self._is_dynamic = is_dynamic
        self._regularizer = regularizer

    def build(self, input_shape):
        embedding_shape, label_shape = input_shape
        self._w = self.add_weight(name='W',shape=(embedding_shape[-1], self._n_classes),
                                  initializer='glorot_uniform',
                                  trainable=True,
                                  regularizer=self._regularizer)
        if self._is_dynamic:
            self._s = self.add_weight(name='S',shape=(),
                                      initializer=Constant(self._init_s),
                                      trainable=False,
                                      aggregation=tf.VariableAggregation.MEAN)

    def call(self, inputs, training=None):
        embedding, label = inputs
        label = tf.cast(label, dtype=tf.int32)

        # Squeezing is necessary for Keras. It expands the dimension to (n, 1)
        label = tf.reshape(label, [-1])

        # Normalize features and weights and compute dot product
        x = tf.nn.l2_normalize(embedding, axis=1)
        w = tf.nn.l2_normalize(self._w, axis=0)
        logits = tf.matmul(x, w)

        # Fixed AdaCos
        is_dynamic = self._is_dynamic#tf_utils.constant_value(self._is_dynamic)
        if not is_dynamic:
            # _s is not created since we are not in dynamic mode
            output = tf.multiply(self._init_s, logits)
            return output

        training = _resolve_training(self, training)
        if not training:
            # We don't have labels to update _s if we're not in training mode
            return self._s * logits
        else:
            theta = tf.math.acos(
                    K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
            one_hot = tf.one_hot(label, depth=self._n_classes)
            b_avg = tf.where(one_hot < 1.0,
                             tf.exp(self._s * logits),
                             tf.zeros_like(logits))
            b_avg = tf.reduce_mean(tf.reduce_sum(b_avg, axis=1))
            theta_class = tf.gather_nd(
                    theta,
                    tf.stack([
                        tf.range(tf.shape(label)[0]),
                        tf.cast(label, tf.int32)
                    ], axis=1))
            mid_index = tf.shape(theta_class)[0] // 2 + 1
            theta_med = tf.nn.top_k(theta_class, mid_index).values[-1]

            # Since _s is not trainable, this assignment is safe. Also,
            # tf.function ensures that this will run in the right order.
            self._s.assign(
                    tf.math.log(b_avg) /
                    tf.math.cos(tf.minimum(math.pi/4, theta_med)))

            # Return scaled logits
            return self._s * logits

In [8]:
# Function to get our f1 score
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

# Function to combine predictions
def combine_predictions(row):
    if CONCAT:
        x = np.concatenate([row['image_predictions'], row['text_predictions']])
    else:
        x = np.intersect1d(row['image_predictions'], row['text_predictions'])
    return ' '.join( np.unique(x) )

# Function to read out dataset
def read_dataset():
    if GET_CV:
        df = pd.read_csv('../input/shopee-product-matching/train.csv')
        tmp = df.groupby(['label_group'])['posting_id'].unique().to_dict()
        df['matches'] = df['label_group'].map(tmp)
        df['matches'] = df['matches'].apply(lambda x: ' '.join(x))
        if CHECK_SUB:
            df = pd.concat([df, df], axis = 0)
            df.reset_index(drop = True, inplace = True)
        df_cu = cudf.DataFrame(df)
        image_paths = '../input/shopee-product-matching/train_images/' + df['image']
    else:
        df = pd.read_csv('../input/shopee-product-matching/test.csv')
        df_cu = cudf.DataFrame(df)
        image_paths = '../input/shopee-product-matching/test_images/' + df['image']
        
    return df, df_cu, image_paths

# Function to decode our images
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image

# Function to read our test image and return image
def read_image(image):
    image = tf.io.read_file(image)
    image = decode_image(image)
    return image

# Function to get our dataset that read images
def get_dataset(image):
    dataset = tf.data.Dataset.from_tensor_slices(image)
    dataset = dataset.map(read_image, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

# Arcmarginproduct class keras layer
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

# Function to get the embeddings of our images with the fine-tuned model
def get_image_embeddings(image_paths,weight_path):
    embeds = []
    K.clear_session()
    adacos=AdaCos(N_CLASSES, name='AdaCos' )
    inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
    label = tf.keras.layers.Input(shape = (), name = 'inp2')
    x = efn.EfficientNetB5(weights = None, include_top = False)(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = adacos((x,label),False)#margin([x, label])
    output = tf.keras.layers.Softmax(dtype='float32')(x)

    model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
    #model.summary()
    #model.load_weights('../input/shopee-effnet-arcface/EfficientNetB5_arcface_512_42 (1).h5')
    model.load_weights(weight_path)
    model = tf.keras.models.Model(inputs = model.input[0], outputs = model.layers[image_output_layer].output)
    chunk = 5000
    iterator = np.arange(np.ceil(len(df) / chunk))
    for j in iterator:
        a = int(j * chunk)
        b = int((j + 1) * chunk)
        image_dataset = get_dataset(image_paths[a:b])
        image_embeddings = model.predict(image_dataset)
        embeds.append(image_embeddings)
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings
def get_image_embeddingsb3(image_paths,weight_path):
    embeds = []
    K.clear_session()
    adacos=AdaCos(N_CLASSES, name='AdaCos' )
    inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
    label = tf.keras.layers.Input(shape = (), name = 'inp2')
    x = efn.EfficientNetB3(weights = None, include_top = False)(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = adacos((x,label),False)#margin([x, label])
    output = tf.keras.layers.Softmax(dtype='float32')(x)

    model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
    #model.summary()
    #model.load_weights('../input/shopee-effnet-arcface/EfficientNetB5_arcface_512_42 (1).h5')
    model.load_weights(weight_path)
    model = tf.keras.models.Model(inputs = model.input[0], outputs = model.layers[image_output_layer].output)
    chunk = 5000
    iterator = np.arange(np.ceil(len(df) / chunk))
    for j in iterator:
        a = int(j * chunk)
        b = int((j + 1) * chunk)
        image_dataset = get_dataset(image_paths[a:b])
        image_embeddings = model.predict(image_dataset)
        embeds.append(image_embeddings)
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings
def get_image_embeddingsb4(image_paths,weight_path):
    embeds = []
    K.clear_session()
    adacos=AdaCos(N_CLASSES, name='AdaCos' )
    inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
    label = tf.keras.layers.Input(shape = (), name = 'inp2')
    x = efn.EfficientNetB4(weights = None, include_top = False)(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = adacos((x,label),False)#margin([x, label])
    output = tf.keras.layers.Softmax(dtype='float32')(x)

    model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
    #model.summary()
    #model.load_weights('../input/shopee-effnet-arcface/EfficientNetB5_arcface_512_42 (1).h5')
    model.load_weights(weight_path)
    model = tf.keras.models.Model(inputs = model.input[0], outputs = model.layers[image_output_layer].output)
    chunk = 5000
    iterator = np.arange(np.ceil(len(df) / chunk))
    for j in iterator:
        a = int(j * chunk)
        b = int((j + 1) * chunk)
        image_dataset = get_dataset(image_paths[a:b])
        image_embeddings = model.predict(image_dataset)
        embeds.append(image_embeddings)
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings

# Return tokens, masks and segments from a text array or series


In [9]:
from transformers import AutoTokenizer,BertTokenizer
from sklearn.preprocessing import LabelEncoder
tokenizer = BertTokenizer.from_pretrained('../input/bert-tokenizer/tokenizer')
#tokenizer.save_pretrained('tokenizer')

In [10]:
TFM_PATH = 'bert-base-uncased'
from transformers import AutoTokenizer, TFAutoModel ,TFBertModel
class RobertaArcFace(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.roberta = TFAutoModel.from_pretrained(TFM_PATH)
        adacos=AdaCos(N_CLASSES, name='AdaCos' ) 
        self.arc_margin =adacos
        self.softmax = tf.keras.layers.Softmax(dtype='float32')
    def call(self, inputs):
        tokens, masks, labels = inputs
        out = self.roberta(tokens, masks)
        feats = out.last_hidden_state[:, 0, :]
        out = self.arc_margin((feats, labels),False)
        out = self.softmax(out)
        return out
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

def tokenize(df):
    inputs = tokenizer(df.title.tolist(), return_tensors='tf', max_length=64, padding='max_length', truncation=True)
    return inputs['input_ids'].numpy(), inputs['attention_mask'].numpy()
def load_df():
    if GET_CV:
        test_df = pd.read_csv('../input/shopee-product-matching/train.csv')
    else:
        test_df = pd.read_csv('../input/shopee-product-matching/test.csv')
    #test_df = pd.read_csv('../input/shopee-product-matching/train.csv')
    #test_df['label_group'] = LabelEncoder().fit_transform(test_df['label_group'])
    #N_CLASSES = test_df['label_group'].nunique()
    #train_x, valid_x = train_test_split(train_df[['title', 'label_group']], shuffle=True, stratify=train_df['label_group'], random_state=SEED, test_size=0.33)
    return test_df[['title']]#, 'label_group']]#train_x, valid_x
def load_ds(tokens, masks, labels):
    text_ds = tf.data.Dataset.from_tensor_slices((tokens, masks, labels))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    ds = tf.data.Dataset.zip((text_ds, label_ds))
    return ds
def load():
    test_df = load_df()
    test_x = tokenize(test_df)
    #test_ds['input_ids']=test_x[0]
    #test_ds['attention_mask']=test_x[1]
    #test_ds = load_ds(*test_x, test_df.label_group.values)
    return test_x[0],test_x[1]#,test_df['label_group'].to_numpy()
# Function to get our text title embeddings using a pre-trained bert model
def get_text_embeddings(text_model_weight_path,max_len = 64):
    embeds = []
    K.clear_session()
    #module_url = "../input/shopee-external-models/bert_en_uncased_L-24_H-1024_A-16_1"
    #bert_layer = hub.KerasLayer(module_url, trainable = True)
    #vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    #do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
    #tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
    #text = bert_encode(df['title'].values, tokenizer, max_len = max_len)

    text=load()
    K.clear_session()
    roberta=TFBertModel.from_pretrained('../input/base-bert-model/base_bert')
    adacos=AdaCos(N_CLASSES, name='AdaCos' ) 
    softmax= tf.keras.layers.Softmax(dtype='float32')
    tokens = tf.keras.layers.Input(shape = (max_len,), dtype=tf.int32, name = 'tokens')
    masks  = tf.keras.layers.Input(shape = (max_len,), dtype=tf.int32, name = 'masks')
    labels = tf.keras.layers.Input(shape = (), name = 'label')
    out = roberta(tokens, masks)
    feats = out.last_hidden_state[:, 0, :]
    out = adacos((feats, labels),False)
    out = softmax(out)
    model = tf.keras.models.Model(inputs = [tokens,masks, labels], outputs = [out])
    #model = RobertaArcFace()#([tokens,masks,labels])
    model.load_weights(text_model_weight_path)
    model = tf.keras.models.Model(inputs = model.input[0:2], outputs = model.layers[-4].output)
    chunk = 5000
    iterator = np.arange(np.ceil(len(text[0]) / chunk))
    for j in iterator:
        a = int(j * chunk)
        b = int((j + 1) * chunk)
        text_chunk = ((text[0][a:b], text[1][a:b]))
        text_embeddings = model.predict(text_chunk, batch_size = BATCH_SIZE)
        embeds.append(text_embeddings)
    del model
    text_embeddings = np.concatenate(embeds)
    print(f'Our text embeddings shape is {text_embeddings.shape}')
    del embeds
    gc.collect()
    return text_embeddings
    
# Function to get 50 nearest neighbors of each image and apply a distance threshold to maximize cv
def get_simple_text_embeddings(max_features = 25_000):
    model = TfidfVectorizer(stop_words = 'english', binary = True, max_features = max_features)
    text_embeddings = model.fit_transform(df_cu['title']).toarray()
    del model
    gc.collect()
    return text_embeddings

In [11]:
df, df_cu, image_paths = read_dataset()
df[['posting_id']].to_csv('submission.csv', index = False)

In [12]:
from tensorflow.keras import backend as K

image_embeddings0 = get_image_embeddings(image_paths,image_model_weight_sam)
image_embeddings1 = get_image_embeddingsb3(image_paths,image_model_weight_path1)
image_embeddings2 = get_image_embeddingsb4(image_paths,image_model_weight_path2)
#image_embeddings3 = get_image_embeddings(image_paths,image_model_weight_path3)
image_embeddings_list=[image_embeddings0,image_embeddings1,image_embeddings2]#,image_embeddings1,image_embeddings2]#[image_embeddings0,image_embeddings1,image_embeddings2,image_embeddings3]


Our image embeddings shape is (3, 2048)
Our image embeddings shape is (3, 1536)
Our image embeddings shape is (3, 1792)


In [13]:
from copy import deepcopy
#save=deepcopy(image_embeddings_list)

In [14]:
#image_model_weight_path0
#model = RobertaArcFace()
text_embeddings = get_text_embeddings(text_model_weight_path0)
gc.collect()

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at ../input/base-bert-model/base_bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Our text embeddings shape is (3, 768)


11370

In [15]:
#image_text_embeddings=(image_embeddings+text_embeddings*0.4)/2

In [16]:
K.clear_session()
#print(text_embeddings2)

In [17]:
def range_sigmoid(x,range):
    #if  x<0:
    #    return 0
    p=np.log(3)*4/range#return 0.75 when x=range/4 
    y=p*x
    return np.exp(y)/(1+np.exp(y))

In [18]:
def get_text_predictions(df, max_features = 25_000):
    
    model = TfidfVectorizer(stop_words = 'english', binary = True, max_features = max_features)
    text_embeddings = model.fit_transform(df_cu['title']).toarray()
    preds = []
    CHUNK = 1024*4

    print('Finding similar titles...')
    CTS = len(df)//CHUNK
    if len(df)%CHUNK!=0: CTS += 1
    for j in range( CTS ):

        a = j*CHUNK
        b = (j+1)*CHUNK
        b = min(b,len(df))
        print('chunk',a,'to',b)

        # COSINE SIMILARITY DISTANCE
        cts = cupy.matmul( text_embeddings, text_embeddings[a:b].T).T

        for k in range(b-a):
            IDX = cupy.where(cts[k,]>0.75)[0]
            o = df.iloc[cupy.asnumpy(IDX)].posting_id.values
            preds.append(o)
    
    del model,text_embeddings
    gc.collect()
    return preds

In [19]:
from scipy.sparse import lil_matrix
def query_expansion( embeddings, KNN = 3,image = True):
    model = NearestNeighbors(n_neighbors = KNN)
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)
    res=deepcopy(embeddings)
    #res.reshape(res.shape[0],1,res.shape[1])
    matrix_length=embeddings.shape[0]
    for i in range(matrix_length):
        res[i]+=sum(embeddings[indices[i]])
    res/=(KNN+1)
    return res
def get_neighbors(df, embeddings,best_thresholds, KNN = 50, image = True,test=False):
    model = NearestNeighbors(n_neighbors = KNN)
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)
    #print(distances.shape)
    #best_thresholds_image=4.5
    #best_thresholds_text=7.5
    matrix_length=embeddings.shape[0]
    predictions_score=lil_matrix((matrix_length,matrix_length))#np.zeros((embeddings.shape[0],embeddings.shape[0]))
    # Iterate through different thresholds to maximize cv, run this in interactive mode, then replace else clause with a solid threshold
    if test:#GET_CV:
        if image:
            thresholds = list(np.arange(2.6,3.4, 0.2))
        else:
            thresholds = list(np.arange(5, 15, 1))
        scores = []
        for threshold in thresholds:
            predictions = []
            for k in range(embeddings.shape[0]):
                idx = np.where(distances[k,] < threshold)[0]
                #predictions_score[k]+=
                ids = indices[k,idx]
                posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
                predictions.append(posting_ids)
            df['pred_matches'] = predictions
            df['f1'] = f1_score(df['matches'], df['pred_matches'])
            score = df['f1'].mean()
            print(f'Our f1 score for threshold {threshold} is {score}')
            scores.append(score)
        thresholds_scores = pd.DataFrame({'thresholds': thresholds, 'scores': scores})
        max_score = thresholds_scores[thresholds_scores['scores'] == thresholds_scores['scores'].max()]
        best_threshold = max_score['thresholds'].values[0]
        best_score = max_score['scores'].values[0]
        print(f'Our best score is {best_score} and has a threshold {best_threshold}')
        
        # Use threshold
        """
        predictions = []
        for k in tqdm(range(embeddings.shape[0])):
            if image:
                best_thresholds=best_thresholds_image
            else:
                best_thresholds=best_thresholds_text
            idx = np.where(distances[k,] < best_thresholds)[0]
            #for i in idx:
            #predictions_score[k,idx]+= np.frompyfunc(range_sigmoid,distances[k,idx]-best_thresholds,best_thresholds)
            for i in idx:
                predictions_score[k,i]=range_sigmoid(best_thresholds-distances[k,i],best_thresholds)
            ids = indices[k,idx]
            posting_ids = df['posting_id'].iloc[ids].values
            predictions.append(posting_ids)
        """
        return predictions_score
    
    # Because we are predicting the test set that have 70K images and different label groups, confidence should be smaller
    else:
        res_len=0
        predictions = []
        for k in tqdm(range(embeddings.shape[0])):
            #if image:
            #    best_thresholds=best_thresholds_image
            #else:
            #    best_thresholds=best_thresholds_text
            idx = np.where(distances[k,] < best_thresholds*1.25)[0]
            res_len+=len(idx)
            #ids = indices[k,idx]
            #predictions_score[k,idx]= np.frompyfunc(range_sigmoid,distances[k,idx]-best_thresholds,best_thresholds)
            for i in idx:
                predictions_score[k,indices[k,i]]=range_sigmoid(best_thresholds-distances[k,i],best_thresholds)
            #ids = indices[k,idx]
            #posting_ids = df['posting_id'].iloc[ids].values
            #predictions.append(posting_ids)
        #same_idx=np.where(prefictions_score > 0.5)[0]
        #posting_ids = df['posting_id'].iloc[indices[k,same_idx]].values
        print(res_len)
    del model, distances, indices
    gc.collect()
    return df, predictions_score

In [20]:
#image_embeddings_list=save.deepcopy()

In [21]:
# Get neighbors for image_embeddings

image_scores_list=[]
th_list=[0.6,1.8,3]
#th_list=[0.8,2.1,4]
for i, em in enumerate(image_embeddings_list):
    expanded_em=query_expansion(em,KNN=2)
    df, image_scores = get_neighbors(df, expanded_em,th_list[i], KNN =100, image = True)
    image_scores_list.append(image_scores)
#df, image_scores = get_neighbors(df, image_text_embeddings,4.5, KNN =50, image = True)

ValueError: n_neighbors must be <= number of samples in index

In [22]:
# Get neighbors for image_embeddings
#image_scores_list=[]
#th_list=[0.9,2.4,4.6]
#i=1
#df, image_scores = get_neighbors(df, image_embeddings_list[i],th_list[i], KNN =100, image = True,test=True)
#image_scores_list.append(image_scores)

In [23]:
# Get neighbors for text_embeddings
th_text=7#10.5
expanded_em=query_expansion(text_embeddings,KNN=2)
df,text_scores = get_neighbors(df, expanded_em,th_text, KNN = 100, image = False)

ValueError: n_neighbors must be <= number of samples in index

In [24]:
#df,text_scores2 = get_neighbors(df,text_embeddings2,1, KNN = 100, image = False)

In [25]:
def tostring_predictions(x):
    return ' '.join( np.unique(x) )

In [26]:
#print(image_scores.getrow(1).nonzero())
#print(image_scores.getrow(1).toarray())
#print(np.where(image_scores.getrow(1).toarray() >0.5))

In [27]:
res_scores=text_scores.tocsr()
#res_scores+=image_scores.tocsr()
weights=[0.35,0.325,0.325]
for i in range(3):
    res_scores+=image_scores_list[i].tocsr()*weights[i]
res_scores/=2

NameError: name 'text_scores' is not defined

In [28]:
#res_scores=image_scores_list[0].tocsr()

In [29]:
res_predictions = []
#res_scores+=image_scores_list[2].tocsr()*0.33
#for image_scores in image_scores_list:
#    res_scores+=image_scores.tocsr()
#1+len(image_scores_list)
for i in range(res_scores.shape[0]):
    arr=res_scores.getrow(i).toarray()
    idx=np.where(arr>0.35)[1]
    posting_ids = df['posting_id'].iloc[idx].values
    res_predictions.append(tostring_predictions(posting_ids))

NameError: name 'res_scores' is not defined

In [30]:
#print(res_scores.getrow(3))


In [31]:
# Concatenate image predctions with text predictions

if GET_CV:
    #df['image_predictions'] = image_predictions
    #df['text_predictions'] = text_predictions
    df['pred_matches'] = res_predictions
    df['f1'] = f1_score(df['matches'], df['pred_matches'])
    score = df['f1'].mean()
    print(f'Our final f1 cv score is {score}')
    #df['matches'] = df['pred_matches']
    df[['posting_id', 'pred_matches']].to_csv('submission.csv', index = False)
else:
    #df['image_predictions'] = image_predictions
    #df['text_predictions'] = text_predictions
    df['matches'] =  res_predictions##df.apply(combine_predictions, axis = 1)
    df[['posting_id', 'matches']].to_csv('submission.csv', index = False)

ValueError: Length of values (0) does not match length of index (3)

In [32]:
df.head(20)

,posting_id,image,image_phash,title
0,test_2255846744,0006c8e5462ae52167402bac1c2e916e.jpg,ecc292392dc7687a,Edufuntoys - CHARACTER PHONE ada lampu dan mus...
1,test_3588702337,0007585c4d0f932859339129f709bfdc.jpg,e9968f60d2699e2c,(Beli 1 Free Spatula) Masker Komedo | Blackhea...
2,test_4015706929,0008377d3662e83ef44e1881af38b879.jpg,ba81c17e3581cabe,READY Lemonilo Mie instant sehat kuah dan goreng
